# Cybersecurity advanced
## Lab 08: Siem



Links to Syllabus

- [Theoretical](https://hogenttin.github.io/cybersecurity-advanced/lesson-8/theory/)
- [Practical](https://hogenttin.github.io/cybersecurity-advanced/lesson-8/lab/)

# Demo

`ssh -L 443:172.30.0.20:443 wazuh`

localhost:443

inlog wazuh dashboard:

- login admin

- password: kU.53gP2LHhJ0U94+yl3GkIK8RHq+NqX


New process = auditd, not implemented

